In [2]:
! pip install spacy scikit-learn
! python -m spacy download en_core_web_sm


2023-11-27 05:59:06.298523: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-27 05:59:06.298592: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-27 05:59:06.298625: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-27 05:59:07.563415: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 39.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [19]:
import pandas as pd
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Load spaCy model
nlp = spacy.load("en_core_web_sm")

# Preprocessing function using spaCy
def preprocess(text):
    doc = nlp(text)
    return " ".join([token.lemma_ for token in doc if not token.is_stop and not token.is_punct])

def classify_news(subject, df, method='randomforest'):
    subject_df = df[df['subject'] == subject]

    #subject_df['processed_summary'] = subject_df['summary'].apply(preprocess)
    subject_df['processed_title'] = subject_df['title'].apply(preprocess)
    #subject_df['processed_description'] = subject_df['description'].apply(preprocess)

    # Combine 'subject' and 'processed_summary' as features
    subject_df['features'] = subject_df['processed_title']

    # Feature extraction using TF-IDF
    tfidf = TfidfVectorizer(max_features=1000)
    X = tfidf.fit_transform(subject_df['features'])
    y = subject_df['action']

    # Splitting the dataset into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Selecting the model based on the method argument
    if method == 'randomforest':
        model = RandomForestClassifier()
    elif method == 'logistic':
        model = LogisticRegression()
    elif method == 'svm':
        model = SVC()
    elif method == 'knn':
        model = KNeighborsClassifier()
    elif method == 'decisiontree':
        model = DecisionTreeClassifier()
    else:
        raise ValueError("Unknown method provided.")
    model.fit(X_train, y_train)

    # Model evaluation
    y_pred = model.predict(X_test)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    return report['accuracy'], report['macro avg']['support']


In [20]:
import pandas as pd

# Assuming classify_news is defined as in the previous script
# ...

df = pd.read_csv('news_price.csv')
# Drop rows where 'summary', 'subject', or 'action' are NaN
df = df.dropna(subset=['summary', 'subject', 'action'])

methods = ['randomforest', 'logistic', 'svm', 'knn', 'decisiontree']
results = []

for method in methods:
    for subject in df['subject'].unique():
        print(f"Processing subject: {subject} with method: {method}")
        try:
            accuracy, support = classify_news(subject, df, method=method)
            results.append({
                'subject': subject,
                'accuracy': accuracy,
                'sample': support,
                'method': method,
            })
        except Exception as e:
            print(f"Error processing subject {subject} with method {method}: {e}")

results_df = pd.DataFrame(results)
results_df = results_df.sort_values(by=['accuracy', 'method'], ascending=[False, True])
results_df.to_csv('classify_news_results.csv')
print(results_df.head())


Processing subject: Calendar of Events with method: randomforest


<ipython-input-19-47af0dd23cab>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subject_df['processed_title'] = subject_df['title'].apply(preprocess)
<ipython-input-19-47af0dd23cab>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subject_df['features'] = subject_df['processed_title']


Processing subject: Health with method: randomforest


<ipython-input-19-47af0dd23cab>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subject_df['processed_title'] = subject_df['title'].apply(preprocess)
<ipython-input-19-47af0dd23cab>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subject_df['features'] = subject_df['processed_title']


Processing subject: Company Announcement with method: randomforest


...

<ipython-input-19-47af0dd23cab>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subject_df['processed_title'] = subject_df['title'].apply(preprocess)
<ipython-input-19-47af0dd23cab>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subject_df['features'] = subject_df['processed_title']
<ipython-input-19-47af0dd23cab>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: 

Processing subject: Exchange Members with method: decisiontree
Processing subject: Changes in company's own shares with method: decisiontree


<ipython-input-19-47af0dd23cab>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subject_df['processed_title'] = subject_df['title'].apply(preprocess)
<ipython-input-19-47af0dd23cab>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subject_df['features'] = subject_df['processed_title']
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter 

Processing subject: Pre-Release Comments with method: decisiontree
Processing subject: Changes in share capital and votes with method: decisiontree
Processing subject: Feature Article with method: decisiontree
Error processing subject Feature Article with method decisiontree: With n_samples=1, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.
Processing subject: Exchange announcement with method: decisiontree
Error processing subject Exchange announcement with method decisiontree: With n_samples=1, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.
Processing subject: Company Regulatory Filings with method: decisiontree
Error processing subject Company Regulatory Filings with method decisiontree: With n_samples=1, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.
Processing subject: 

<ipython-input-19-47af0dd23cab>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subject_df['processed_title'] = subject_df['title'].apply(preprocess)
<ipython-input-19-47af0dd23cab>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subject_df['features'] = subject_df['processed_title']
<ipython-input-19-47af0dd23cab>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: 

In [21]:
# Analyze the data to find the method that gives the most results with accuracy above 60%
filtered_df = results_df[results_df['accuracy'] > 0.6]
method_counts = filtered_df['method'].value_counts()

# Determine the method with the most results above 60% accuracy
best_method = method_counts.idxmax()
best_method_count = method_counts.max()

best_method, best_method_count

('randomforest', 16)